In [1]:
import mlflow



import sys
sys.path.append('../src')
from config_loader import load_config
from mlflow_setup import setup_mlflow
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

Общие функции

In [2]:
def extract_topics(model, vectorizer, n_words=10):
    feature_names = vectorizer.get_feature_names_out()
    topics = []
    for topic in model.components_:
        top_words_idx = topic.argsort()[:-n_words-1:-1]
        topics.append([feature_names[i] for i in top_words_idx])
    return topics

def calculate_coherence(topics, texts):
    tokenized_texts = [str(text).split() for text in texts]
    coherence_model = CoherenceModel(
        topics=topics,
        texts=tokenized_texts,
        dictionary=Dictionary(tokenized_texts),
        coherence='c_v'
    )
    return coherence_model.get_coherence()


LDA 1

In [5]:
# Загружаем конфигурацию LDA
cfg = load_config("lda_1")

# Устанавливаем эксперимент
setup_mlflow(cfg)  

with mlflow.start_run(run_name="lda_topic_model") as run:
    
    # 1. Логируем параметры
    mlflow.log_params({
        "vectorizer_max_features": cfg['vectorizer']['max_features'],
        "vectorizer_ngram_range": str(cfg['vectorizer']['ngram_range']),
        "vectorizer_max_df": cfg['vectorizer']['max_df'],
        "vectorizer_min_df": cfg['vectorizer']['min_df'],
        "lda_n_components": cfg['lda']['n_components'],
        "lda_max_iter": cfg['lda']['max_iter'],
        "lda_learning_method": cfg['lda']['learning_method'],
        "lda_random_state": cfg['lda']['random_state'],
    
    # evaluation тоже на корневом уровне
    "evaluation_top_words": cfg['evaluation']['top_words_count']
    })
    
    # Загружаем данные
    print(f"Загрузка данных из {cfg['data']['input']['path']}")
    df_processed = pd.read_csv(
        cfg['data']['input']['path'],
        sep=cfg['data']['input']['delimiter'],
        encoding=cfg['data']['input']['encoding']
    )
    
    # Инициализация и обучение векторизатора
    count_vectorizer = CountVectorizer(
        max_features = cfg['vectorizer']['max_features'],
        ngram_range = tuple(cfg['vectorizer']['ngram_range']),
        max_df = cfg['vectorizer']['max_df'],
        min_df = cfg['vectorizer']['min_df'],
    )
    
    texts = df_processed['message_clean_no_stopwords'].fillna('')
    dataset = count_vectorizer.fit_transform(texts)
    
    # Инициализация и обучение LDA
    lda = LDA( 
        n_components=cfg['lda']['n_components'],
        max_iter=cfg['lda']['max_iter'],    
        learning_method=cfg['lda']['learning_method'],
        evaluate_every=cfg['lda']['evaluate_every'],
        random_state=cfg['lda']['random_state'],
        verbose=1
    )
    
    lda.fit(dataset)
    print(f"LDA модель обучена: {cfg['lda']['n_components']} тем")
    
    # Извлекаем темы 
    topics = extract_topics(lda, count_vectorizer, cfg['evaluation']['top_words_count'])
    
    # Рассчитываем метрики
    coherence = calculate_coherence(topics, texts.tolist())
    
    metrics = {
        "perplexity": lda.perplexity(dataset),
        "log_likelihood": lda.score(dataset),
        "coherence": coherence
    }
    
    mlflow.log_metrics(metrics)
    print(f"Когерентность модели: {coherence:.4f}")
    print(f"Perplexity: {lda.perplexity(dataset):.2f}")
    
    # Логируем модели
    mlflow.sklearn.log_model(count_vectorizer, "count_vectorizer")
    mlflow.sklearn.log_model(lda, "lda_model1")
    
    # Выводим и логируем темы
    print("\nТемы модели:")
    for idx, topic_words in enumerate(topics, 1):
        print(f"Тема {idx}: {', '.join(topic_words)}")
    
    # Логируем темы как артефакт
    topics_content = "\n".join(
        [f"Тема {idx}: {', '.join(words)}" for idx, words in enumerate(topics, 1)]
    )
    mlflow.log_text(topics_content, "topics.txt")
    
    print(f"LDA модель залогирована в MLflow. Run ID: {run.info.run_id}")

Загружаю конфиг из: ../config/lda_1.yaml
✓ Tracking URI: http://127.0.0.1:8080
✓ Experiment: topic_modeling
✓ Готово к запуску нового эксперимента
Загрузка данных из ../data/data_processed2.csv
iteration: 1 of max_iter: 50
iteration: 2 of max_iter: 50
iteration: 3 of max_iter: 50
iteration: 4 of max_iter: 50
iteration: 5 of max_iter: 50, perplexity: 11.9279
iteration: 6 of max_iter: 50
iteration: 7 of max_iter: 50
iteration: 8 of max_iter: 50
iteration: 9 of max_iter: 50
iteration: 10 of max_iter: 50, perplexity: 11.5192
iteration: 11 of max_iter: 50
iteration: 12 of max_iter: 50
iteration: 13 of max_iter: 50
iteration: 14 of max_iter: 50
iteration: 15 of max_iter: 50, perplexity: 11.3988
iteration: 16 of max_iter: 50
iteration: 17 of max_iter: 50
iteration: 18 of max_iter: 50
iteration: 19 of max_iter: 50
iteration: 20 of max_iter: 50, perplexity: 11.3897
LDA модель обучена: 7 тем
Когерентность модели: 0.5987


2025/12/25 03:59:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/25 03:59:51 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Perplexity: 11.39


2025/12/25 03:59:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/25 03:59:55 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



Темы модели:
Тема 1: улица, остановка, автобус, маршрут, быть, путь, работать, центр, движение, линия
Тема 2: движение, центр, улица, быть, работать, путь, поездка, маршрут, автобус, линия
Тема 3: поездка, путь, быть, работать, движение, центр, маршрут, улица, автобус, поезд
Тема 4: подслушать, путь, поезд, автобус, линия, движение, быть, работать, остановка, маршрут
Тема 5: маршрут, быть, автобус, работать, остановка, путь, движение, улица, центр, линия
Тема 6: поезд, движение, путь, быть, остановка, маршрут, линия, поездка, работать, автобус
Тема 7: линия, работать, путь, движение, поезд, быть, маршрут, центр, остановка, автобус
LDA модель залогирована в MLflow. Run ID: bd00ff727c6141c388f53a20c4c158a5
🏃 View run lda_topic_model at: http://127.0.0.1:8080/#/experiments/4/runs/bd00ff727c6141c388f53a20c4c158a5
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/4


LDA 2

In [9]:
# Загружаем конфиг
cfg = load_config("lda_2")

# Настраиваем эксперимент
setup_mlflow(cfg)  

with mlflow.start_run(run_name=f"lda_{cfg['lda']['n_components']}topics") as run:
    
    # Загружаем артефакты из MLflow
    client = mlflow.tracking.MlflowClient()
    
    # Загружаем векторизатор
    vectorizer_path = client.download_artifacts(
        cfg['data']['vectorizer']['run_id'], 
        "count_vectorizer"
    )
    vectorizer = mlflow.sklearn.load_model(vectorizer_path)
    
    # Загружаем данные
    data_path = client.download_artifacts(
        cfg['data']['data']['run_id'], 
        "data/data_processed2/data_processed2.csv"
    )
    df = pd.read_csv(data_path)
    
    # Логируем параметры
    mlflow.log_params({
        "n_topics": cfg['lda']['n_components'],
        "vectorizer_source": cfg['data']['vectorizer']['run_id'],
        "data_source": cfg['data']['data']['run_id'],
        "max_iter": cfg['lda']['max_iter']
    })
    
    # Подготавливаем данные
    texts = df['message_clean_no_stopwords'].fillna('')
    dataset = vectorizer.transform(texts)
    
    # Обучаем LDA
    lda = LDA(
        n_components=cfg['lda']['n_components'],
        max_iter=cfg['lda']['max_iter'],    
        learning_method=cfg['lda']['learning_method'],
        evaluate_every=cfg['lda']['evaluate_every'],
        random_state=cfg['lda']['random_state'],
        verbose=1,
        n_jobs=-1
    )
    lda.fit(dataset)

    topics = extract_topics(lda, vectorizer, n_words=10)
    
    # Считаем метрики
    coherence = calculate_coherence(topics, texts.tolist())
    
    mlflow.log_metrics({
        "perplexity": lda.perplexity(dataset),
        "log_likelihood": lda.score(dataset),
        "coherence": coherence
    })
    
    # Сохраняем модель
    mlflow.sklearn.log_model(lda, "lda_model2")
    
    # Выводим темы
    print("\nТемы:")
    for i, topic_words in enumerate(topics, 1):
        print(f"Тема {i}: {', '.join(topic_words)}")
    
    # Сохраняем темы
    with open("topics.txt", "w") as f:
        for i, words in enumerate(topics, 1):
            f.write(f"Тема {i}: {', '.join(words)}\n")
    mlflow.log_artifact("topics.txt")
    
    print(f"LDA модель залогирована в MLflow. Run ID: {run.info.run_id}")

Загружаю конфиг из: ../config/lda_2.yaml
✓ Tracking URI: http://127.0.0.1:8080
✓ Experiment: topic_modeling
✓ Готово к запуску нового эксперимента


iteration: 1 of max_iter: 100
iteration: 2 of max_iter: 100
iteration: 3 of max_iter: 100
iteration: 4 of max_iter: 100
iteration: 5 of max_iter: 100, perplexity: 14.2321
iteration: 6 of max_iter: 100
iteration: 7 of max_iter: 100
iteration: 8 of max_iter: 100
iteration: 9 of max_iter: 100
iteration: 10 of max_iter: 100, perplexity: 14.0795
iteration: 11 of max_iter: 100
iteration: 12 of max_iter: 100
iteration: 13 of max_iter: 100
iteration: 14 of max_iter: 100
iteration: 15 of max_iter: 100, perplexity: 14.1199


2025/12/25 04:25:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/25 04:25:59 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!



Темы:
Тема 1: остановка, маршрут, быть, улица, автобус, путь, работать, движение, линия, центр
Тема 2: движение, улица, быть, центр, маршрут, поездка, путь, работать, автобус, остановка
Тема 3: поездка, быть, движение, центр, маршрут, путь, поезд, улица, работать, линия
Тема 4: движение, быть, путь, поездка, маршрут, поезд, центр, линия, автобус, работать
Тема 5: автобус, быть, улица, маршрут, работать, остановка, движение, путь, линия, центр
Тема 6: путь, поезд, движение, быть, поездка, остановка, маршрут, линия, автобус, улица
Тема 7: линия, быть, маршрут, движение, путь, автобус, поезд, поездка, работать, остановка
Тема 8: остановка, автобус, улица, работать, быть, маршрут, центр, движение, путь, поезд
Тема 9: поезд, линия, быть, движение, маршрут, работать, поездка, подслушать, путь, остановка
Тема 10: центр, движение, быть, поездка, улица, маршрут, путь, работать, автобус, линия
Тема 11: маршрут, быть, автобус, работать, путь, улица, движение, линия, поездка, остановка
Тема 12: у

LDA 3 (новый векторизатор + новый датасет)

In [11]:
# Загружаем конфиг
cfg = load_config("lda_3")
setup_mlflow(cfg)  

with mlflow.start_run(run_name=f"lda_{cfg['lda']['n_components']}topics_new") as run:
    
    # Загружаем данные
    client = mlflow.tracking.MlflowClient()
    data_path = client.download_artifacts(
        cfg['data']['data_run_id'], 
        "output_data/data_without_stopwords.csv"
    )
    df = pd.read_csv(data_path)
    
    # Векторизатор
    vectorizer = CountVectorizer(
        max_features=cfg['vectorizer']['max_features'], 
        ngram_range=tuple(cfg['vectorizer']['ngram_range']),
        max_df=cfg['vectorizer']['max_df'], 
        min_df=cfg['vectorizer']['min_df']
    )
    
    # Подготовка данных
    texts = df['message_clean_no_stopwords'].fillna('')
    dataset = vectorizer.fit_transform(texts)
    texts_list = texts.tolist()
    
    # Логируем параметры
    mlflow.log_params({
        "n_topics": cfg['lda']['n_components'],
        "data_source": cfg['data']['data_run_id'],
        "vectorizer_max_features": cfg['vectorizer']['max_features'],
        "vectorizer_ngram_range": str(cfg['vectorizer']['ngram_range']),
        "vectorizer_max_df": cfg['vectorizer']['max_df'],
        "vectorizer_min_df": cfg['vectorizer']['min_df'],
        "lda_max_iter": cfg['lda']['max_iter']
    })
    
    # LDA
    lda = LDA(
        n_components=cfg['lda']['n_components'],
        max_iter=cfg['lda']['max_iter'],    
        learning_method=cfg['lda']['learning_method'],
        evaluate_every=cfg['lda']['evaluate_every'],
        random_state=cfg['lda']['random_state'],
        verbose=1,
        n_jobs=-1
    )
    lda.fit(dataset)

    # Извлекаем темы
    topics = extract_topics(lda, vectorizer, n_words=10)
    
    # Преобразуем биграммы для когерентности
    valid_topics = []
    for topic in topics:
        words = []
        for phrase in topic:
            words.extend(str(phrase).split())
        # Убираем дубликаты
        unique_words = list(dict.fromkeys(words))
        valid_topics.append(unique_words[:10])
    
    # Когерентность
    if len(texts_list) > 2000:
        texts_list = texts_list[:2000]
    
    try:
        coherence = calculate_coherence(valid_topics, texts_list)
    except:
        coherence = 0.0
    
    # Метрики
    mlflow.log_metrics({
        "perplexity": lda.perplexity(dataset),
        "coherence": coherence,
        "log_likelihood": lda.score(dataset)
    })
    
    # Сохраняем
    mlflow.sklearn.log_model(vectorizer, "vectorizer")
    mlflow.sklearn.log_model(lda, "lda_model")
    
    # Темы
    topics_text = "\n".join([f"Тема {i}: {', '.join(words)}" 
                           for i, words in enumerate(topics, 1)])
    mlflow.log_text(topics_text, "topics.txt")
    
    # Результаты
    print(f"Темы: {cfg['lda']['n_components']}")
    print(f"Perplexity: {lda.perplexity(dataset):.2f}")
    print(f"Coherence: {coherence:.4f}")
    print(f"Run ID: {run.info.run_id}")

Загружаю конфиг из: ../config/lda_3.yaml
✓ Tracking URI: http://127.0.0.1:8080
✓ Experiment: topic_modeling
✓ Готово к запуску нового эксперимента


iteration: 1 of max_iter: 100
iteration: 2 of max_iter: 100
iteration: 3 of max_iter: 100
iteration: 4 of max_iter: 100
iteration: 5 of max_iter: 100, perplexity: 424.9296
iteration: 6 of max_iter: 100
iteration: 7 of max_iter: 100
iteration: 8 of max_iter: 100
iteration: 9 of max_iter: 100
iteration: 10 of max_iter: 100, perplexity: 396.5783
iteration: 11 of max_iter: 100
iteration: 12 of max_iter: 100
iteration: 13 of max_iter: 100
iteration: 14 of max_iter: 100
iteration: 15 of max_iter: 100, perplexity: 392.7948
iteration: 16 of max_iter: 100
iteration: 17 of max_iter: 100
iteration: 18 of max_iter: 100
iteration: 19 of max_iter: 100
iteration: 20 of max_iter: 100, perplexity: 392.1191
iteration: 21 of max_iter: 100
iteration: 22 of max_iter: 100
iteration: 23 of max_iter: 100
iteration: 24 of max_iter: 100
iteration: 25 of max_iter: 100, perplexity: 391.8181
iteration: 26 of max_iter: 100
iteration: 27 of max_iter: 100
iteration: 28 of max_iter: 100
iteration: 29 of max_iter: 100


2025/12/25 04:30:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/25 04:30:55 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2025/12/25 04:31:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/12/25 04:31:01 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


Темы: 15
Perplexity: 387.14
Coherence: 0.4703
Run ID: 9c87e1b8b13446ac9d420d611f900312
🏃 View run lda_15topics_new at: http://127.0.0.1:8080/#/experiments/4/runs/9c87e1b8b13446ac9d420d611f900312
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/4
